<div style="border: 1px solid black">
<b><center><font size="4">Data Science Toolkit</font></center></b>

<b><center><font size="2">Docker</font></center></b>

<b><center><font size="1">4 - Docker Orquestration</font></center></b>
</div>

**Developed by**: [Ricardo Campos](https://www.di.ubi.pt/~rcampos)<br>
**email:**  ricardo.campos@ubi.pt<br>
**Affiliation:** *Assistant Professor* @ [University of Beira Interior](http://www.ubi.pt);
*Researcher* @ [LIAAD](https://www.inesctec.pt/en/centres/liaad)-[INESC TEC](https://www.inesctec.pt/en)

<hr>

<p><a href="4-DockerCompose-Orquestration.ipynb" title="Download Notebook" download><img src="https://www.di.ubi.pt/~rcampos/assets/img_tutorials/download.jpg" align = "left" width="50" height="50" alt="Download Notebook"></a></p>
<p>&nbsp;</p>
<p>&nbsp;</p>

<hr>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Docker-Compose-Orquestration" data-toc-modified-id="Docker-Compose-Orquestration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Docker Compose Orquestration</a></span></li><li><span><a href="#What-is-Docker-Compose" data-toc-modified-id="What-is-Docker-Compose-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>What is Docker Compose</a></span></li><li><span><a href="#Docker-Compose-File-(.yaml)" data-toc-modified-id="Docker-Compose-File-(.yaml)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Docker Compose File (.yaml)</a></span></li><li><span><a href="#Run" data-toc-modified-id="Run-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Run</a></span></li></ul></div>

<hr>

## Docker Compose Orquestration

<div class="alert alert-success alertsuccess" style="margin-top: 20px">

## Objetivos de aprendizagem  <a class="tocSkip">
    
No final deste notebook o aluno deverá ter conhecimentos básicos sobre orquestração em docker.


## Learning Objectives  <a class="tocSkip">
       
When concluding this notebook, the student should have basic concepts of docker orquestration. </div>

<div class="alert alert-success alertsuccess" style="margin-top: 20px">

## Sumário  <a class="tocSkip">
### Orquestração em Docker <a class="tocSkip">

Introdução dos alunos à orquestração em docker.
- Conceito de orquestração;
- Criação de um ficheiro de orquestração;
- Execução de um ficheiro de orquestração.
    
## Class Summary  <a class="tocSkip">
### Docker Orquestration <a class="tocSkip">
Introduction to docker orquestration.
- Notions of orquestration;
- Creating a docker orquestration file;
- Running a docker orquestration file.

</div>

<hr>

## What is Docker Compose

Compose is a tool for defining and running multi-container Docker applications as a single service. With Compose, you use a YAML file to configure your application’s services. Then, with a single command, you create and start all the services from your configuration.

<img src="https://www.di.ubi.pt/~rcampos/assets/img_tutorials/DataScience/DockerCompose.jpg" width="300" height="300" align="center"/>

For example, suppose you had an application which required ElasticSearch and Kibana (a free and open user interface that lets you visualize your Elasticsearch data), you could create one file which would start both the containers as a service without the need to start each one separately.

Using Compose is basically a three-step process:
- Define your app’s environment with a Docker compose file so it can be reproduced anywhere.
- Define the services that make up your app in `docker-compose.yml` so they can be run together in an isolated environment.
- Run `docker-compose up` and Compose starts and runs your entire app.
- You can also run `docker-compose up -d` which will run in dettacted mode.

`docker-compose up --build --detach` builds the images if the images do not exist and starts the containers in detached mode.

Now suppose the system needs to re-build an image which makes use of a python package that has suffered an update. It happens that upon building Docker will likely make use of a cached version. To re-build it from the scratch you should use the following commands:

```python
docker-compose build --no-cache
docker-compose up -detach
```

## Docker Compose File (.yaml)

Based on: https://quoeamaster.medium.com/deploying-elasticsearch-and-kibana-with-docker-86a4ac78d851

Begin by creating a folder (e.g., `H:\Backup\Research\Programming\CodePython\MyDocker\ElasticSearch`) which will hold both the `docker-compose.yml` and the `data` folder (which you should also create by hand).

All Docker Compose files are YAML (Ain’t Markup Language) files. In the following we show how to download and run ElasticSearch and Kibana with a single command. Now let’s go ahead and create our first Docker Compose file (`docker-compose.yml`). Save it under the given folder (e.g., `H:\Backup\Research\Programming\CodePython\MyDocker\ElasticSearch`). Note que os nomes das imagens e dos containers deverão ser minúsculas.

```
version: "3.8"
services:
  
  elastic_search:
    image: docker.elastic.co/elasticsearch/elasticsearch:7.16.1
    container_name: elastic_search
    restart: always
    environment:
      - discovery.type=single-node
      - "ES_JAVA_OPTS=-Xms512m -Xmx512m"
    ulimits:
      memlock:
        soft: -1
        hard: -1
    volumes:
      - es_data:/usr/share/elasticsearch/data
    ports:
      - "9200:9200"
    networks:
      - es-network
  
  kibana:
    image: docker.elastic.co/kibana/kibana:7.16.1
    container_name: kibana
    restart: always
    ports:
      - "5601:5601"
    networks:
      - es-network
    environment:
      ELASTICSEARCH_URL: http://elastic_search:9200
      ELASTICSEARCH_HOSTS: http://elastic_search:9200
    depends_on:
      - elastic_search

volumes:
  es_data:
    driver: local
    driver_opts:
      type: 'none'
      o: 'bind'
      device: 'h:\Backup\Research\Programming\CodePython\MyDocker\ElasticSearch\data'

networks:
  es-network:
    driver: bridge
```

Let’s take a close look at the various details of this file:

- Apart from specifying the images, we are mapping the ports of the containers to the ports on the host machine. Elasticsearch runs on port 9200 and Kibana on port 5601 so we are mapping both these ports to the corresponding ports on the host machine.
- A `single-node Elasticsearch cluster` is created.
- Finally, we mount the folder `/usr/share/elasticsearch/data` where elastic search data is stored to the local volume `es_data`. More precisaly data will be kept in the folder: `h:\Backup\Research\Programming\JupyterNotebooks\Python\Docker\ElasticSearch\data`. In case you are using docker toolbox specify the following path instead: `/c/Users/campo/Desktop/ElasticSearch/data`.

You can also add `PYTHONUNBUFFERED=1` environment variable to each service of the docker-compose whenever you are using a service that should print log messages to the console (such as with Flask APIs).

```python
environment:
      - PYTHONUNBUFFERED=1
```

## Run

Thus, the first thing to do (in your docker machine) is to change the working directory to the one where you have the `yaml` file. In this example, we assume the following path:

```
h:
cd H:\Backup\Research\Programming\CodePython\MyDocker\ElasticSearch
```

Next, try to start our setup with the above settings by running the following command under the folder where you have your docker compose file:

```python
docker-compose up --build --detach
```

This will start pulling the images from docker registry and create the containers. This may take a while depending on whether you already have the images on your machine or not and also depending on your internet speed.

After that you can log on elastic search as follows:

<u>docker desktop</u>:

`ElasticSearch`: http://localhost:9200/

`kibana`: http://localhost:5601